-Next step: relate job satisfaction to (un)fair treatment
-Does being treated unfairly affect job satisfaction? Is this relation dependent on the level of negative reciprocity?

Again, first read in relevant datasets and variables: reciprocity measures from 2005 and outcome and controls from 2006.

In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

In [97]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
data_folder = Path("/Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw")
# define relevant subsets of SOEP-data, vp: 2005, wp: 2006
file_names = ['vp', 'wp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [98]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_05 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})

In [99]:
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear","wp12402", "wp12401","wp0102", "wp5902", 'wp43b01', "wp43b03", "wp43b05", "wp43b07"]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={"wp12401": "gender", "wp12402": "year_birth", 'wp43b01': "recog_sup", "wp43b03": "recog_effort", "wp43b05": "recog_personal", "wp43b07": "recog_pay", "wp5902": "wage_lastmonth", "wp0102": "satisfaction_work"})

In [101]:

hours06 = pd.read_stata(file_paths_2[1], columns=["pid","hid", "syear", "wvebzeit", "betr06", "wpsbil", "nace06", "werwzeit", "wbilzeit"]).set_index(['pid', 'hid'])
work06 = hours06.rename(columns={'wvebzeit': 'working_hours', "betr06": "firmsize", "wpsbil": "school_degree", "nace06": "sector", "werwzeit": "tenure" , "wbilzeit" : "years_educ"})

In [102]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}

# mapping for sectors: only to easier remove negatives
sector_map = {
    "[1] Landwirtschaft und  Jagd": 1,
    "[2] Forstwirtschaft": 2,
    "[5] Fischerei und Fischzucht": 5,
    "[10] Kohlenbergbau, Torfgewinnung": 10,
    "[11] Gewinnung von Erdöl und Erdgas, Erbringung damit verbundener Dienstleistungen": 11,
    "[12] Bergbau auf Uran- und Thoriumerze": 12,
    "[13] Erzbergbau": 13,
    "[14] Gewinnung von Steinen und Erden, sonstiger Bergbau": 14,
    "[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken": 15,
    "[16] Tabakverarbeitung": 16,
    "[17] Herstellung von Textilien": 17,
    "[18] Herstellung von Bekleidung": 18,
    "[19] Herstellung von Leder und Lederwaren": 19,
    "[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)": 20,
    "[21] Herstellung von Papier, Pappe und Waren daraus": 21,
    '[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern': 22,
    "[23] Kokerei, Mineralölverarbeitung, Herstellung und Verarbeitung von Spalt- und Brutstoffen": 23,
    "[24] Herstellung von chemischen Erzeugnissen": 24,
    "[25] Herstellung von Gummi- und Kunststoffwaren": 25,
    "[26] Herstellung von Glas und Glaswaren, Keramik, Verarbeitung von Steinen und Erden": 26,
    "[27] Metallerzeugung und -bearbeitung": 27,
    "[28] Herstellung von Metallerzeugnissen": 28,
    "[29] Maschinenbau": 29,
    "[31] Herstellung von Geräten der Elektrizitätserzeugung, -verteilung u. Ä.": 31,
    "[30] Herstellung von Büromaschinen, Datenverarbeitungsgeräten und -einrichtungen": 30,
    "[32] Rundfunk- und Nachrichtentechnik": 32,
    "[33] Medizin-, Mess-, Steuer- und Regelungstechnik, Optik, Herstellung von Uhren": 33,
    "[34] Herstellung von Kraftwagen und Kraftwagenteilen": 34,
    "[35] Sonstiger Fahrzeugbau": 35,
    "[36] Herstellung von Möbeln, Schmuck, Musikinstrumenten, Sportgeräten, Spielwaren und sonstigen Erzeugnissen": 36,
    "[37] Rückgewinnung": 37,
    "[40] Energieversorgung": 40,
    "[41] Wasserversorgung": 41,
    "[45] Bau": 45,
    "[50] Kraftfahrzeughandel; Instandhaltung und Reparatur von Kraftfahrzeugen; Tankstellen": 50,
    "[51] Handelsvermittlung und Großhandel (ohne Handel mit Kraftfahrzeugen)": 51,
    "[52] Einzelhandel (ohne Handel mit Kraftfahrzeugen und ohne Tankstellen); Reparatur von Gebrauchsgütern": 52,
    "[55] Beherbergungs- und Gaststätten": 55,
    "[60] Landverkehr; Transport in Rohrfernleitungen": 60,
    "[61] Schifffahrt": 61,
    "[62] Luftfahrt": 62,
    "[63] Hilfs- und Nebentätigkeiten für den Verkehr; Verkehrsvermittlung": 63,
    "[64] Nachrichtenübermittlung": 64,
    "[65] Kreditinstitute": 65,
    "[66] Versicherungen (ohne Sozialversicherung)": 66,
    "[67] Mit den Kreditinstituten und Versicherungen verbundene Tätigkeiten": 67,
    "[70] Grundstücks- und Wohnungswesen": 70,
    "[71] Vermietung beweglicher Sachen ohne Bedienungspersonal": 71,
    "[72] Datenverarbeitung und Datenbanken": 72,
    "[73] Forschung und Entwicklung": 73,
    "[74] Erbringung von unternehmensbezogenen Dienstleistungen": 74,
    "[75] Öffentliche Verwaltung, Verteidigung, Sozialversicherung": 75,
    "[80] Erziehung und Unterricht": 80,
    "[85] Gesundheits-, Veterinär- und Sozialwesen": 85,
    "[90] Abwasser- und Abfallbeseitigung und sonstige Entsorgung": 90,
    "[91] Interessenvertretungen sowie kirchliche und sonstige Vereinigungen (ohne Sozialwesen, Kultur und Sport)": 91,
    "[92] Kultur, Sport und Unterhaltung": 92,
    "[93] Erbringung von sonstigen Dienstleistungen": 93,
    "[95] Private Haushalte mit Hauspersonal": 95,					
    "[96] Industrie - ohne weitere Zuordnung": 96,					
    "[97] Handwerk - ohne weitere Zuordnung": 97,					
    "[98] Dienstleistungen ohne weitere Zuordnung": 98,					
    "[99] Exterritoriale Organisationen und Körperschaften": 99,				
    "[100] Produzierendes Gewerbe ohne w.Zuordnung": 100,
    "[-1] keine Angabe": -1,
    '[-2] trifft nicht zu': -2, 
    "[-3] unplausibler Wert": -3,
    "[-4] unzulaessige Mehrfachantwort": -4, 
    "[-5] in Fragebogenversion nicht enthalten": -5,
    "[-6] Fragebogenversion mit geaenderter Filterfuehrung": -6, 
    "[-7] nur in weniger eingeschraenkter Edition verfuegbar": -7,
    "[-8] Frage in diesem Jahr nicht Teil des Frageprogramms": -8,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}

In [103]:
def recode_categoricals(inputdf):

    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    merged['sector'] = merged['sector'].map(sector_map)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    merged['satisfaction_work'] = merged['satisfaction_work'].map(satisfaction_mapping)
    output = merged
    return output

In [104]:
# Merge dataframes: a bit tough to read as its nested, merges 3 dataframes: 2005,2006,and 2006gen
allmerged_df = pd.merge(pd.merge(df_05, df_06, on=['pid', 'hid']), work06, on=['pid', 'hid'])
recoded = recode_categoricals(allmerged_df).astype('int')
# construct industry-relative wage
sector_wage_averages = recoded.groupby('sector')['wage_lastmonth'].mean()
recoded["sector_avg_wage"] = recoded["sector"].map(sector_wage_averages)
recoded["relative_wage"] = recoded["wage_lastmonth"] / recoded["sector_avg_wage"]
# replaces negative values with n.a.n 
recoded = recoded.mask(recoded < 0, np.nan) 
# construct avg reciprocity measure
recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
# construct age, potential experience and age^2
recoded['age'] = 2006 - recoded['year_birth']
recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
recoded["age_squared"] = (recoded["age"] ** 2) / 100
# recode categoricals back to make it better readable
#recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
#recoded["sector"]=recoded["sector"].map(reversed_mapping_sector)
#recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

# transform binary variables with 1 and 2 into 1 and 0
columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "satisfaction_work"]

# Iterate over the columns and replace the values 2 with 0
for col in columns_to_transform:
    recoded[col] = recoded[col].replace({2: 0})

# save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
dfnan = recoded


dfnan.head(3)

,,syear_x,take_revenge,similar_problems,insult_back,syear_y,year_birth,gender,satisfaction_work,wage_lastmonth,recog_sup,...,school_degree,sector,tenure,years_educ,sector_avg_wage,relative_wage,avg_rec,age,potential_experience,age_squared
pid,hid,,,,,,,,,,,,,,,,,,,,,
201,27,2005,1.0,5.0,1.0,2006,1926,0,NaN,NaN,NaN,...,1.0,NaN,NaN,10.0,94.115693,NaN,2.333333,80,3844,64.00
203,60313,2005,2.0,3.0,2.0,2006,1960,1,1.0,2300.0,0.0,...,4.0,72.0,1.0,18.0,2196.757962,1.046997,2.333333,46,784,21.16
602,60,2005,5.0,4.0,3.0,2006,1958,0,NaN,200.0,1.0,...,4.0,80.0,1.0,18.0,1773.037897,0.112801,4.000000,48,900,23.04


In [83]:
dfnan.columns

Index(['syear_x', 'take_revenge', 'similar_problems', 'insult_back', 'syear_y',
       'year_birth', 'gender', 'satisfaction_work', 'wage_lastmonth',
       'recog_sup', 'recog_effort', 'recog_personal', 'recog_pay', 'syear',
       'working_hours', 'firmsize', 'school_degree', 'sector', 'tenure',
       'years_educ', 'sector_avg_wage', 'relative_wage', 'avg_rec', 'age',
       'potential_experience', 'age_squared'],
      dtype='object')

### Mincer Wage Regression

In [108]:
df_mincer = dfnan.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y',
       'recog_personal', 'recog_pay', 'syear_y', 'year_birth','sector_avg_wage', 'school_degree', 'recog_sup', 'age', 'relative_wage', 'recog_effort', 'working_hours', 'syear', 'satisfaction_work'])

In [110]:
# Convert 'gender' and 'sector' columns to categorical data type
for col in ['gender', 'sector']:
    df_mincer[col] = df_mincer[col].astype('category')

In [111]:
df_mincer = df_mincer.dropna()

In [112]:
# Define the dependent variable
y = df_mincer['wage_lastmonth']

# Define the independent variables
X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the Mincer wage regression model
mincer_model = sm.OLS(y, X).fit()

# Print the summary statistics of the model
#mincer_model.summary()

# Get the residuals of the model
residuals_mincer = mincer_model.resid

In [118]:
# drop all columns that we dont need for the regression.
df_regression = dfnan.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'recog_effort',
       'recog_personal', 'recog_pay', 'syear_y', 'year_birth', 'sector_avg_wage', 'age', 'sector', 'wage_lastmonth', 'school_degree', "syear"])

In [119]:
# Create a new column in the dataframe with the same name as the residuals array
df_regression['relative_wage'] = None

# Match the rows of the dataframe with the values in the residuals array using the index
df_regression.loc[df_regression.index, 'relative_wage'] = residuals_mincer

# Rename the column "relative_wage" to "mincer_residuals"
df_regression = df_regression.rename(columns={'relative_wage': 'mincer_residuals'})

/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_68974/553673764.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_regression.loc[df_regression.index, 'relative_wage'] = residuals_mincer


In [120]:
#again, add interaction term reciprocity x unfair treatment 
df_regression["interaction"] = df_regression["recog_sup"] * df_regression["avg_rec"]

In [121]:
#drop nans
df_regression = df_regression.dropna()


In [122]:
from statsmodels.discrete.discrete_model import Probit

Y = df_regression["satisfaction_work"]
X = df_regression.drop(columns=["satisfaction_work"])
X = sm.add_constant(X)
model = Probit(Y, X.astype(float))
probit_model = model.fit()

probit_model.summary()

Optimization terminated successfully.
         Current function value: 0.483417
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:      satisfaction_work   No. Observations:                 6832
Model:                         Probit   Df Residuals:                     6820
Method:                           MLE   Df Model:                           11
Date:                Fri, 06 Jan 2023   Pseudo R-squ.:                 0.09232
Time:                        16:13:05   Log-Likelihood:                -3302.7
converged:                       True   LL-Null:                       -3638.6
Covariance Type:            nonrobust   LLR p-value:                5.928e-137
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.9143      0.181      5.043      0.000       0.559       1.270
gender                   0.1468      0.043      3.452      0.001       0.063       0.230
recog_sup               -0.7751      0.087     -8.905      0.000      -0.946      -0.604
working_hours           -0.0049      0.002     -2.034      0.042      -0.010      -0.000
firmsize                 0.0196      0.007      2.787      0.005       0.006       0.033
tenure                   0.0070      0.002      3.171      0.002       0.003       0.011
years_educ               0.0424      0.007      6.073      0.000       0.029       0.056
mincer_residuals         0.0002   3.07e-05      6.413      0.000       0.000       0.000
avg_rec                 -0.0582      0.017     -3.478      0.001      -0.091      -0.025
potential_experience     0.0001      0.000      0.420      0.674      -0.000       0.001
age_squared             -0.0178      0.019     -0.954      0.340      -0.054       0.019
interaction             -0.0064      0.025     -0.258      0.796      -0.055       0.042
========================================================================================
"""

negative interaction: combination of unfair treatment and negative reciprocity is associated with lower levels of job satisfaction, however insignificant

Now, instead of using a probit model, we use OLS.

In [123]:
from statsmodels.regression.linear_model import OLS

Y = df_regression["satisfaction_work"]
X = df_regression.drop(columns=["satisfaction_work"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit()

lpm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      satisfaction_work   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     67.61
Date:                Fri, 06 Jan 2023   Prob (F-statistic):          2.37e-144
Time:                        16:13:13   Log-Likelihood:                -3369.4
No. Observations:                6832   AIC:                             6763.
Df Residuals:                    6820   BIC:                             6845.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.8121      0.049     16.740      0.000       0.717       0.907
gender                   0.0339      0.011      2.998      0.003       0.012       0.056
recog_sup               -0.2043      0.025     -8.294      0.000      -0.253      -0.156
working_hours           -0.0012      0.001     -1.806      0.071      -0.002       0.000
firmsize                 0.0057      0.002      2.959      0.003       0.002       0.009
tenure                   0.0019      0.001      3.171      0.002       0.001       0.003
years_educ               0.0101      0.002      5.581      0.000       0.007       0.014
mincer_residuals      4.378e-05   7.27e-06      6.019      0.000    2.95e-05     5.8e-05
avg_rec                 -0.0132      0.004     -3.063      0.002      -0.022      -0.005
potential_experience  3.096e-05   8.62e-05      0.359      0.719      -0.000       0.000
age_squared             -0.0047      0.005     -0.929      0.353      -0.015       0.005
interaction             -0.0115      0.007     -1.630      0.103      -0.025       0.002
==============================================================================
Omnibus:                      974.221   Durbin-Watson:                   1.922
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1455.444
Skew:                          -1.128   Prob(JB):                         0.00
Kurtosis:                       2.861   Cond. No.                     9.26e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [124]:
latex_output = lpm_model.summary().as_latex()
print(latex_output)

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}        & satisfaction\_work & \textbf{  R-squared:         } &     0.098   \\
\textbf{Model:}                &        OLS         & \textbf{  Adj. R-squared:    } &     0.097   \\
\textbf{Method:}               &   Least Squares    & \textbf{  F-statistic:       } &     67.61   \\
\textbf{Date:}                 &  Fri, 06 Jan 2023  & \textbf{  Prob (F-statistic):} & 2.37e-144   \\
\textbf{Time:}                 &      16:14:17      & \textbf{  Log-Likelihood:    } &   -3369.4   \\
\textbf{No. Observations:}     &         6832       & \textbf{  AIC:               } &     6763.   \\
\textbf{Df Residuals:}         &         6820       & \textbf{  BIC:               } &     6845.   \\
\textbf{Df Model:}             &           11       & \textbf{                     } &             \\
\textbf{Covariance Type:}      &     nonrobust      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tab